### 1. FastText로 영화 리뷰 훈련: Embedding vector로 단어별 16 벡터로 변환

### 2. 감성분석

### 데이터 다운로드

In [1]:
import pandas as pd
import requests

# res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

# with open('ratings_train.txt', 'wb') as f:
#     f.write(res.content)

nsmc = pd.read_csv('ratings_train.txt', sep='\t')

In [3]:
nsmc

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


### 전처리

In [4]:
import re

def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [5]:
nsmc.loc[0, 'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [6]:
find_hangul(nsmc.loc[0, 'document'])

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [15]:
nsmc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
nsmc['document'].notnull()

0         True
1         True
2         True
3         True
4         True
          ... 
149995    True
149996    True
149997    True
149998    True
149999    True
Name: document, Length: 150000, dtype: bool

In [9]:
nsmc[nsmc['document'].notnull()]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [16]:
nsmc[nsmc['document'].notnull()]['document']

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [19]:
# null 행을 제외한 데이터에서 한글만 추출
data = nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)

In [20]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [21]:
data[1]

['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나']

In [22]:
def only_hangul(text):
    return ' '.join(find_hangul(text))

In [24]:
only_hangul(nsmc.loc[0, 'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [27]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [28]:
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

In [29]:
data2[1]

'흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나'

In [30]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data2))

### FastText 모형 학습

In [31]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

In [32]:
# sb=0(기본값. cbow 학습: ->?<-) / 1(skip-gram 학습: ?<-->?)
# alpha=0.025(학습률)
# min_alpha(학습률을 낮춰주는 역할)
# window=5(좌우 몇개 단어 맥락으로 학습을 할지 결정)
# min_count=5(최소 몇 번 나와야 학습을 함)
# vector_size=100(기본값)

model = FastText(vector_size=16)

In [33]:
model.build_vocab(corpus_iterable=data)

In [34]:
model.train(corpus_iterable=data,
epochs=5,
total_examples=model.corpus_count,
total_words=model.corpus_total_words)

(3999601, 5829395)

### 저장

In [35]:
model.save('nsmc.fasttext')

### 임베딩

In [36]:
model = FastText.load('nsmc.fasttext')

In [37]:
model.wv.key_to_index

{'영화': 0,
 '너무': 1,
 '정말': 2,
 '진짜': 3,
 '이': 4,
 '그냥': 5,
 '왜': 6,
 '이런': 7,
 '더': 8,
 '점': 9,
 '수': 10,
 '영화를': 11,
 '다': 12,
 '잘': 13,
 '좀': 14,
 '보고': 15,
 'ㅋㅋ': 16,
 '그': 17,
 '영화가': 18,
 '영화는': 19,
 '본': 20,
 '봤는데': 21,
 '최고의': 22,
 '아': 23,
 '이건': 24,
 '내가': 25,
 '드라마': 26,
 '없는': 27,
 '없다': 28,
 '평점': 29,
 '완전': 30,
 '이렇게': 31,
 '참': 32,
 '이거': 33,
 '그리고': 34,
 '이게': 35,
 '좋은': 36,
 '있는': 37,
 '연기': 38,
 '내': 39,
 '평점이': 40,
 '보는': 41,
 '최고': 42,
 '다시': 43,
 '역시': 44,
 '스토리': 45,
 '쓰레기': 46,
 'ㅋ': 47,
 '난': 48,
 '많이': 49,
 '것': 50,
 '한': 51,
 'ㅋㅋㅋ': 52,
 '재밌게': 53,
 '없고': 54,
 '또': 55,
 '하는': 56,
 '아깝다': 57,
 '꼭': 58,
 '보면': 59,
 '마지막': 60,
 '가장': 61,
 '뭐': 62,
 '영화다': 63,
 '무슨': 64,
 '하지만': 65,
 '같은': 66,
 'ㅎㅎ': 67,
 '와': 68,
 '별로': 69,
 '작품': 70,
 '솔직히': 71,
 '끝까지': 72,
 '볼': 73,
 '넘': 74,
 '안': 75,
 '대한': 76,
 '만든': 77,
 '봐도': 78,
 '그래도': 79,
 '시간': 80,
 '같다': 81,
 '전혀': 82,
 '좋다': 83,
 '말이': 84,
 '지금': 85,
 '별': 86,
 '아주': 87,
 '근데': 88,
 '중': 89,
 '뭔가': 90,
 '영화의': 91,
 '하

In [38]:
'히어로' in model.wv.key_to_index

True

In [39]:
'슈퍼히어로' in model.wv.key_to_index

False

In [40]:
model.wv['슈퍼히어로'] # 단어 집합에 없더라도, FastText의 경우 n-gram으로 단어를 잘라 학습하기 때문에, 단어의 유사도를 계산할 수 있음

array([-0.19449769,  0.30387518,  0.22050212,  0.17775355,  0.2743885 ,
        0.06692418, -0.21594803, -0.2512149 ,  0.17368731,  0.03521266,
        0.0844363 , -0.33816066,  0.03265488, -0.13027968,  0.03641635,
        0.17477015], dtype=float32)

In [41]:
### 유사도

In [42]:
model.wv.similarity('슈퍼히어로', '히어로') # 코사인 유사도(0 별개 ~ 1 유사)

0.98879474

In [43]:
model.wv.similarity('슈퍼히어로', '평론가')

0.7742573

In [45]:
model.wv.most_similar('평론가') # 평론가와 유사한 단어들

[('높은거야', 0.9918226003646851),
 ('점이상은', 0.9907475709915161),
 ('점이나', 0.9907414317131042),
 ('점이라', 0.9904584884643555),
 ('점이야', 0.9896332621574402),
 ('점이라니', 0.9889189600944519),
 ('점대나', 0.9882330298423767),
 ('점대면', 0.9880049824714661),
 ('점대야', 0.9877113699913025),
 ('점이면', 0.9874235987663269)]

### FastText를 이용한 감성분석

In [46]:
from gensim.models.fasttext import FastText

ft = FastText.load('nsmc.fasttext')

In [47]:
import pandas as pd

nsmc = pd.read_csv('ratings_train.txt', sep='\t')

In [48]:
df = nsmc[nsmc['document'].notnull()]

In [49]:
from sklearn.model_selection import train_test_split

doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'], \
    test_size=0.2, random_state=42)

In [50]:
import re

def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [51]:
import numpy as np

x_train = np.zeros((1000, 16)) # 단어를 1000만 잘라사 16벡터로 만들기

In [52]:
doc_train.shape

(119996,)

In [53]:
doc_train

31989            아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다..
63462                    개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군
17518               장하나 한윤찬 제발 이어주세요 말도안되게 왜 설도현과ㅜㅜㅜ
123410                                   애로영화계의 개OOO
104181                               내용이나 그래픽자체가 허접함
                             ...                    
119882      꿈을 꾸는사람 꿈을 이룬사람 돌멩이도 꿈은 있잖아! 꿈과희망을 주는 영화
103696                                레니 할린.. 이게 뭐니?
131936                                    시라노; 연애조작단
146872                                    집중이 쉽지 않다.
121961    엔딩 장면이 좋고 소소한 일상과 고민과 연애가 공감을 주어 재밌게 보았어여~
Name: document, Length: 119996, dtype: object

In [54]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    print(find_hangul(doc))

['아', '꿀잼ㅋ', '친구랑', '봤는데', '너무', '웃겼음', '그리구', '김우빈', '잘생겼다']
['개건의', '졸작', '스릴러', '스릴러라고', '하게에도', '민망하군']
['장하나', '한윤찬', '제발', '이어주세요', '말도안되게', '왜', '설도현과']
['애로영화계의', '개']
['내용이나', '그래픽자체가', '허접함']
['요새', '이런', '영화가', '드물다는게', '안타깝다']
['역사적', '의의를', '떠나서', '재밌습니다', '지금봐도', '손색이', '없습니다']
['핵꿀잼']
['책을', '읽은', '느낌', '사랑의', '을', '보여줬다']
['평점', '말도안돼']
['난', '역시', '천재야', '개월', '전에', '본', '영화', '기억해냈당']
['재밌음']
['살', '조카는', '무한점', '저는', '점', '가끔', '웃기기도', '하고', '단', '안경', '좀', '불편', '약간', '어지러움']
['하아', '암걸릴뻔']
['태어나서', '별점처음', '써본다', '이런게', '스릴러지', '유해진', '진구씨', '연기쩔어', '왜이제', '봤을까', '너무재밌음']
['나의', '계왕원백배', '원기옥', '기원참', '에네르기파', '마광관살포를', '받아랏']
['많은것을', '말하려', '했지만', '한마디도', '하지', '못한', '영화']
['옴니버스', '호러물로', '한국', '호러물중', '최고의', '영화가', '아닐까']
['내', '인생', '최고의', '강시영화']
['아오걍하지마', '뭔고학년도안된아이를가지고장난이야걍민국이하고지아는', '아빠어디가에서만하지', '왜영화까지끌고나와서', '더이미지안좋게하는건데']
['굿']
['년에', '보고', '년에', '다시', '봐도', '역시나', '최고의', '드라마다']
['연출', '내용', '연기', '음악', '조명', '스타일링', '전부', '굿', '그리고', '이연희의', '재발견']
[

In [56]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    for word in find_hangul(doc):
        print(ft.wv[word])

[ 0.544108    2.4242957   1.3362653  -0.67145926  2.5233624   4.066546
 -5.3759255  -3.155939   -0.07614627 -2.9257817   3.097557   -0.27373844
 -4.711964    0.48305944  2.9365869   3.7551155 ]
[-0.2245817   0.21324518  0.04036958  0.16216294  0.47514698  0.19274043
 -0.3968975  -0.3178537   0.27095792  0.03265693  0.12870423 -0.18098953
 -0.2983772  -0.03649952  0.19820538  0.25631776]
[-1.3610821   0.8263087   1.1421565   0.5924076   1.3449934   0.8127111
 -1.1225107  -0.7831233   1.0566328   0.12537973  0.41135532 -1.4030645
 -0.43060717  0.29260716  0.4700612   0.5471712 ]
[-2.798884   -1.858806    1.4511994   2.9016051   1.9263229   4.090046
 -1.1055747  -1.31525     2.2814558  -1.0771072  -0.38144466 -0.78308207
 -6.403928   -0.31190753  2.5378678   2.4436002 ]
[-1.2458663   0.10615388 -2.2331264   3.879599   -0.39236555  1.1395395
 -0.38386104 -4.9606853   1.2752006   2.834472   -0.4563377  -1.1098037
 -4.155198   -0.2971569   1.0038846   0.17630166]
[-0.36989957  0.46108145  0.

In [57]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv]
    if vs:
        print(np.mean(vs, axis=0))

[array([-0.22950406,  0.23130907,  0.07971949,  0.22334802,  0.20525159,
        -0.00666826, -0.1663897 , -0.27967656,  0.11678622,  0.09322082,
         0.01270725, -0.27391127,  0.04773325, -0.15911694,  0.03008895,
         0.15318345], dtype=float32),
 array([-0.5014879 ,  0.31610867,  0.4271059 ,  0.42688707,  0.59972274,
         0.40393668, -0.5091139 , -0.44713625,  0.331135  ,  0.00989504,
         0.1598425 , -0.6793995 , -0.03773394, -0.1266622 ,  0.05658154,
         0.2242198 ], dtype=float32),
 array([-0.07896253,  0.07346814,  0.03584389,  0.03334629,  0.05108469,
         0.02169103, -0.02977767, -0.05523516,  0.03132934, -0.00823118,
        -0.01231903, -0.08420569,  0.01011608, -0.04051301,  0.02324222,
         0.01303296], dtype=float32),
 array([-0.06379714,  0.04560024, -0.00868068,  0.04521219,  0.10182749,
         0.01562043, -0.08674027, -0.05706596,  0.0545782 ,  0.00507267,
        -0.00473787, -0.06693051, -0.07154614, -0.04754924,  0.05134921,
         0

In [61]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv]
    if vs and i == 1:
        print(doc) # 리뷰 하나
        print(len(vs)) # 리뷰 안에 들어있는 단어 수
        print(vs[0]) # 단어 하나당 16개의 벡터. 해당 리뷰의 벡터 = 단어 6개 * 벡터 16
        print(np.mean(vs, axis=0))

개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군
6
[-0.10400003  0.1312917   0.02423394  0.09750541  0.10434029  0.00993345
 -0.02558543 -0.12328044  0.03057311  0.01787078  0.04376185 -0.11555678
  0.05303329 -0.09364056  0.02100045  0.08888968]
[-0.17955023  1.3003737   0.5330267   0.22383189  0.5948415  -0.22708245
 -0.3849455  -0.6885707   0.38989818 -0.09505951 -0.15582769 -1.1552511
  0.48178875 -0.67057425 -0.341694    0.6881191 ]


In [62]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv]
    if vs:
        # print(doc) # 리뷰 하나
        # print(len(vs)) # 리뷰 안에 들어있는 단어 수
        # print(vs[0]) # 단어 하나당 16개의 벡터. 해당 리뷰의 벡터 = 단어 6개 * 벡터 16
        x_train[i,] = np.mean(vs, axis=0)
    else:
        print(doc)

once upon a dream
good
good
it's jjangge movie
Good message


In [64]:
!python -m gensim.scripts.word2vec2tensor --input nsmc.fasttext --output nsmc.fasttext

2022-11-03 17:17:50,848 - word2vec2tensor - INFO - running /home/jeon2/anaconda3/envs/study/lib/python3.10/site-packages/gensim/scripts/word2vec2tensor.py --input nsmc.fasttext --output nsmc.fasttext
2022-11-03 17:17:50,848 - keyedvectors - INFO - loading projection weights from nsmc.fasttext
Traceback (most recent call last):
  File "/home/jeon2/anaconda3/envs/study/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/jeon2/anaconda3/envs/study/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/jeon2/anaconda3/envs/study/lib/python3.10/site-packages/gensim/scripts/word2vec2tensor.py", line 94, in <module>
    word2vec2tensor(args.input, args.output, args.binary)
  File "/home/jeon2/anaconda3/envs/study/lib/python3.10/site-packages/gensim/scripts/word2vec2tensor.py", line 68, in word2vec2tensor
    model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_model_path, binary=

In [65]:
!ls

 2016-10-20.txt
'22-10-25 Text Analytics_Tokenizer 1.ipynb'
'22-10-26 Text Analytics_Tokenizer 2.ipynb'
'22-10-27 Natural Language Processing_Local Representation 1.ipynb'
'22-10-27 Text Analytics_Tokenizer 3.ipynb'
'22-10-28 Natural Language Processing_LSA.ipynb'
'22-10-28 Natural Language Processing_Local Representation 2.ipynb'
'22-10-31 Natural Language Processing_Sentiment Analysis.ipynb'
'22-11-2 Natural Language Processing_Fast Text.ipynb'
'22-11-2 Natural Language Processing_Language Model CNN.ipynb'
 cloud.png
 imdb.xlsx
 neurips.zip
 nsmc.fasttext
 nsmc.fasttext.wv.vectors_ngrams.npy
 nsmc.krs
 nsmc.pkl
 nsmc.txt
 ratings_train.txt
 word_count.csv
 한국어불용어100.txt
